## Load Dependencies

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, random
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adamax
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input, BatchNormalization
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model

## Data Preparation

### Load Dataset

In [2]:
data = pd.read_csv('../datasets/fer2013.csv')
data.shape

(35887, 3)

### Data Exploration

In [3]:
data.head(3)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training


In [4]:
print(data.isnull().mean()*100)

emotion    0.0
pixels     0.0
Usage      0.0
dtype: float64


### Data Visualization

In [23]:
CLASSES_LIST = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']
CLASSES_DICT = {0 : 'Angry', 1 : 'Disgust', 2 : 'Fear', 3 : 'Happiness', 4 : 'Sad', 5 : 'Surprise', 6 : 'Neutral'}

In [24]:
fig = px.bar(
    x = CLASSES_LIST,
    y = [list(data['emotion']).count(i) for i in np.unique(data['emotion'])],
    color = np.unique(data['emotion']),
    color_continuous_scale="Emrld"
)
fig.update_xaxes(title="Emotions")
fig.update_yaxes(title = "Number of Images")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Train Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    })
fig.write_image("../results/visualizations/data_distribution.png", width=3.5*300, height=2*300, scale=4) 
fig.show()

## Data Preprocessing

In [16]:
# Shuffling
data = data.sample(frac=1)

### One Hot Encoding

In [18]:
labels = to_categorical(data[['emotion']], num_classes=7)

features = data['pixels'].astype(str).str.split(' ').tolist()
features = np.uint8(features)

### Standardization

In [20]:
X = features.reshape((35887*2304,1))

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape
X = X.reshape((35887, 48, 48,1))

### Train-Test-Val Split

In [21]:
# Training - 80%
# Test - 10%
# Validation - 10%
X_train, X_temp, y_train, y_temp = train_test_split(X, labels, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## Data Augmentation

In [25]:
data_gen = ImageDataGenerator(
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    zoom_range = 0.2
)
val_gen = ImageDataGenerator(
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    zoom_range = 0.2
)

In [26]:
data_gen.fit(X_train)
val_gen.fit(X_val)

train_gen = data_gen.flow(X_train, y_train, batch_size=64)
val_gen = data_gen.flow(X_val, y_val, batch_size=64)

## Model Training

### Building CNN Model

In [35]:
def model_cnn():
    
    model = Sequential()

    # 1st Batch
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1)))
    model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 2nd Batch
    model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 3rd Batch
    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 4th Batch
    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 5th Batch
    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 6th Batch
    model.add(Flatten()) 
    model.add(Dense(256,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    # 7th Batch
    model.add(Dense(512,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    # 8th Batch
    model.add(Dense(7, activation='softmax'))
    model.compile(
        optimizer = Adam(lr=0.0001), 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])
    
    return model

model = model_cnn()

c:\Users\Arush\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



ValueError: Argument(s) not recognized: {'lr': 0.0001}